# Automated Test Script

## Importing required libraries

In [1]:
import numpy as np
import pandas as pd
import sys, os

### Locating our logs files in our working directory

In [2]:
LOGS_PATH = os.path.join(os.getcwd(), 'logs')

### Replace this with the log file name that your looking to test

In [3]:
LOG_BARCODE = '2018-04-03_21-10'

### Loading DeepEye & Tester log files into Pandas DataFrames

In [4]:
tester_log = pd.read_csv(os.path.join(LOGS_PATH, "[" + LOG_BARCODE + "]--DeepEye.csv"), index_col=0)
deepeye_log = pd.read_csv(os.path.join(LOGS_PATH, "[" + LOG_BARCODE + "]--Tester.csv"), index_col=0) 

#### Let's look at the log files

#### Log file that was created by human testers to express the `GROUND TRUTH`

In [5]:
tester_log

,FRAME_ID,PEDESTRIAN,VEHICLES,BIKES,STOP_SIGN,TRAFFIC_LIGHT,OFF_LANE,COLLISION
0,0,0,0,0,0,0,1,1
1,1,0,0,0,0,0,1,1
2,2,1,1,0,0,0,0,0
3,3,1,1,0,0,0,0,0
4,4,1,1,0,0,0,0,0
5,5,1,1,0,0,0,0,0
6,6,1,1,0,0,0,1,0
7,7,1,1,1,0,1,0,0
8,8,1,1,0,0,1,1,0
9,9,1,1,0,0,1,0,0


#### Auto-generated log by DeepEye

In [6]:
deepeye_log

,FRAME_ID,PEDESTRIAN,VEHICLES,BIKES,STOP_SIGN,TRAFFIC_LIGHT,OFF_LANE,COLLISION
0,0,0,0,0,0,0,1,1
1,1,0,0,1,0,0,1,1
2,2,1,1,0,0,0,0,0
3,3,1,1,0,0,0,1,0
4,4,1,1,1,0,0,0,1
5,5,1,1,0,0,0,0,0
6,6,1,0,0,1,0,1,0
7,7,1,0,1,0,1,0,0
8,8,1,1,0,0,1,1,1
9,9,1,1,0,0,1,0,0


### Compare the two dataframes and identify each feature where the `GROUND TRUTH` and the `PREDICTION` by DeepEye were different

In [7]:
combined = pd.concat([tester_log.set_index('FRAME_ID'), deepeye_log.set_index('FRAME_ID')], 
                   axis='columns', keys=['GROUND TRUTH', 'PREDICTION'])

combined = combined.swaplevel(axis='columns')[tester_log.columns[1:]]
combined

PEDESTRIAN                VEHICLES                   BIKES  \
         GROUND TRUTH PREDICTION GROUND TRUTH PREDICTION GROUND TRUTH   
FRAME_ID                                                                
0                   0          0            0          0            0   
1                   0          0            0          0            0   
2                   1          1            1          1            0   
3                   1          1            1          1            0   
4                   1          1            1          1            0   
5                   1          1            1          1            0   
6                   1          1            1          0            0   
7                   1          1            1          0            1   
8                   1          1            1          1            0   
9                   1          1            1          1            0   
10                  1          1            1          1            1   
11                  1          1            1          1            0   
12                  1          1            1          1            0   
13                  1          1            1          1            0   
14                  0          0            1          1            1   
15                  1          1            0          0            1   
16                  1          1            1          1            1   
17                  0          0            1          1            0   
18                  1          1            1          1            0   
19                  1          1            1          1            1   
20                  0          0            1          0            0   
21                  1          1            1          0            0   
22                  0          0            1          1            0   
23                  0          0            1          0            0   
24                  0          0            1          1            0   
25                  0          1            1          1            0   
26                  0          0            1          1            0   
27                  0          0            1          1            0   
28                  0          1            1          1            0   
29                  0          0            1          1            0   
30                  0          1            1          1            0   
31                  1          1            1          1            0   
32                  0          0            1          0            0   
33                  0          0            1          0            0   

                       STOP_SIGN            TRAFFIC_LIGHT             \
         PREDICTION GROUND TRUTH PREDICTION  GROUND TRUTH PREDICTION   
FRAME_ID                                                               
0                 0            0          0             0          0   
1                 1            0          0             0          0   
2                 0            0          0             0          0   
3                 0            0          0             0          0   
4                 1            0          0             0          0   
5                 0            0          0             0          0   
6                 0            0          1             0          0   
7                 1            0          0             1          1   
8                 0            0          0             1          1   
9                 0            0          0             1          1   
10                1            0          0             0          0   
11                0            0          0             1          1   
12                0            0          0             1          1   
13                0            0          0             1          1   
14                1            0          0             1          1   
15           

In [8]:
log = (tester_log != deepeye_log).stack()
errors = log[log]
errors.index.names = ['FRAME_ID', 'FEATURE']
difference_loc = np.where(tester_log != deepeye_log)
ground_truth = tester_log.values[difference_loc]
prediction = deepeye_log.values[difference_loc]
comparison_table = pd.DataFrame({'GROUND TRUTH': ground_truth, 'PREDICTION': prediction}, index=errors.index)

In [9]:
comparison_table

GROUND TRUTH  PREDICTION
FRAME_ID FEATURE                                
1        BIKES                     0           1
3        OFF_LANE                  0           1
4        BIKES                     0           1
         COLLISION                 0           1
6        VEHICLES                  1           0
         STOP_SIGN                 0           1
7        VEHICLES                  1           0
8        COLLISION                 0           1
14       COLLISION                 0           1
16       BIKES                     1           0
         TRAFFIC_LIGHT             0           1
17       COLLISION                 0           1
20       VEHICLES                  1           0
21       VEHICLES                  1           0
         STOP_SIGN                 0           1
23       VEHICLES                  1           0
         COLLISION                 0           1
25       PEDESTRIAN                0           1
27       COLLISION                 0           1
28       PEDESTRIAN                0           1
30       PEDESTRIAN                0           1
         BIKES                     0           1
         STOP_SIGN                 0           1
         TRAFFIC_LIGHT             0           1
         COLLISION                 0           1
31       COLLISION                 0           1
32       VEHICLES                  1           0
         COLLISION                 1           0
33       VEHICLES                  1           0
         STOP_SIGN                 0           1

### Calculate a score for each frame based on the number of false predictions

In [10]:
scores = comparison_table.groupby(['FRAME_ID']).size().reset_index(name='SCORE')
scores['SCORE'] =  len(tester_log.columns) - scores['SCORE'] - 1
results = deepeye_log.join(scores.set_index('FRAME_ID'), on='FRAME_ID')
results['SCORE'].fillna(len(tester_log.columns) - 1, inplace=True) 
results['SCORE'] = results['SCORE'] / 7
results[['SCORE']].style.format("{:.2%}")

,SCORE
0,100.00%
1,85.71%
2,100.00%
3,85.71%
4,71.43%
5,100.00%
6,71.43%
7,85.71%
8,85.71%
9,100.00%


### Display some basic statistical analytics

In [11]:
pd.DataFrame(results['SCORE'].describe())

,SCORE
count,34.000000
mean,0.873950
std,0.152346
min,0.285714
25%,0.857143
50%,0.857143
75%,1.000000
max,1.000000
